<center> Data Drifts

# PSI visualization

### Принятые границы значений PSI:
- `< 0.1`: Нет значимого дрейфа (данные стабильны)
- `0.1 – 0.25`: Возможен дрейф
- `> 0.25`: Сильный дрейф

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout

In [6]:
def plot_update(func):
    """ 
    Декоратор визуализации
    """
    def wrapper (*args, **kwargs): 
        _, ax = plt.subplots(figsize=(6, 4))
        
        value = func(*args, **kwargs)
        ax.bar(['PSI'], [value], color='skyblue')
        ax.set_ylim(-0.5, 50)
        ax.axhline(0, color='gray', linestyle='--')
        ax.text(0, value + 0.2, f'{value:.4f}', ha='center', va='bottom', fontsize=12, fontweight='bold', color='black')
        plt.show()
    return wrapper

def feature_generator (mean_:  float, std_: float, size: int):
    """ 
    Генератор нормально распределенного ряда значений фичи
    """
    np.random.seed(0)
    return np.random.normal(loc=mean_, scale=std_, size=size,)

def calculate_ratio (data: np.ndarray, bins: np.ndarray) -> np.ndarray:
    """ 
    Вычисляет долю наблюдений в каждом бине с защитой от нулевых значений.
    Параметры:
    ----------
    - data: Одномерный массив данных, для которого нужно рассчитать доли вхождений по бинам.
    - bins: массив бинов (интервалов) для разбиения диапазона значений.

    Возвращает:
    ----------
    - массив нормированных долей по каждому бину, в котором нули заменены на 1e-6 (защита от логарифма нуля).
    """
    counts, _ = np.histogram(data, bins= bins, density=False)
    train_ratio = counts / len(data)
    zero_protected = np.clip(train_ratio, 1e-6, None)
    return zero_protected


@plot_update
def psi (
    n_batches: int,
    train_mean: float, 
    train_std: float, 
    train_size: int,
    current_mean: float, 
    current_std: float, 
    current_size: int
    ):
    """ 
    Расчет PSI по формуле
    """
    train = feature_generator (train_mean, train_std, train_size)
    current = feature_generator (current_mean, current_std, current_size)
    
    bins = np.linspace (
        start = min(min(train), min(current)),
        stop = max(max(train), max(current)),
        num = n_batches+1
    )        
    
    train_ratio = calculate_ratio (train, bins)
    current_ratio = calculate_ratio (current, bins)
    psi_values = (train_ratio - current_ratio) * np.log(train_ratio / current_ratio)

    return np.sum(psi_values)


In [8]:
l = Layout(width='1500px')
style = {'description_width': '200px'}

interact(
    psi,
    n_batches = IntSlider(value=10, min=5, max=100, step=1, description='ЧИСЛО БИНОВ', layout = l, style = style),
    
    train_size = IntSlider(value=1000, min=1000, max=100000, step=100, description='TRAIN: размер выборки', layout = l, style = style),
    current_size = IntSlider(value=1000, min=1000, max=100000, step=100, description='CURRENT: размер выборки', layout = l, style = style),
    
    train_mean = FloatSlider(value=1.0, min=0, max=100.0, step=1.0, description='TRAIN: среднее', layout = l, style = style),
    current_mean = FloatSlider(value=1.0, min=0, max=100.0, step=1.0, description='CURRENT: среднее', layout = l, style = style),
    
    train_std = FloatSlider(value=1.0, min=0.01, max=10.0, step=0.1, description='TRAIN: стандартное отклонение', layout = l, style = style),
    current_std = FloatSlider(value=1.0, min=0.01, max=10.0, step=0.1, description='CURRENT: стандартное отклонение', layout = l, style = style),
)


interactive(children=(IntSlider(value=10, description='ЧИСЛО БИНОВ', layout=Layout(width='1500px'), min=5, sty…

<function __main__.plot_update.<locals>.wrapper(*args, **kwargs)>